In [ ]:
from dotenv import load_dotenv
import os
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime, timedelta

load_dotenv()
API_KEY = os.environ.get("API_KEY")
SECRET_KEY = os.environ.get("SECRET_KEY")


In [14]:
# Create the client
client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

# Define date range
end_date = datetime.now()
start_date = end_date - timedelta(days=14)  # last 5 days of intraday

# Create request for NVDA
request = StockBarsRequest(
    symbol_or_symbols=["NVDA", "AMD"],
    timeframe=TimeFrame.Minute,
    start=start_date,
    end=end_date
)


# Fetch data
bars = client.get_stock_bars(request)

# # Convert to DataFrames
df = bars.df.reset_index()
# df_amd = bars_amd.df.reset_index()

print(df)
# print(df_amd.head())

      symbol                 timestamp      open      high       low  \
0       NVDA 2025-07-25 18:52:00+00:00  173.6700  173.6800  173.5800   
1       NVDA 2025-07-25 18:53:00+00:00  173.5853  173.6200  173.5514   
2       NVDA 2025-07-25 18:54:00+00:00  173.5800  173.5900  173.5300   
3       NVDA 2025-07-25 18:55:00+00:00  173.5500  173.6350  173.5200   
4       NVDA 2025-07-25 18:56:00+00:00  173.6000  173.6700  173.5600   
...      ...                       ...       ...       ...       ...   
18814    AMD 2025-08-08 18:47:00+00:00  171.3850  171.4800  171.3200   
18815    AMD 2025-08-08 18:48:00+00:00  171.4501  171.6699  171.4200   
18816    AMD 2025-08-08 18:49:00+00:00  171.6325  171.8000  171.5700   
18817    AMD 2025-08-08 18:50:00+00:00  171.7895  171.8000  171.4301   
18818    AMD 2025-08-08 18:51:00+00:00  171.4700  171.6700  171.4623   

          close    volume  trade_count        vwap  
0      173.5899  105924.0       1156.0  173.616945  
1      173.5900  204397.0    